<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#All-wikidata-properties" data-toc-modified-id="All-wikidata-properties-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>All wikidata properties</a></span></li><li><span><a href="#Get-wikidata-ID" data-toc-modified-id="Get-wikidata-ID-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get wikidata ID</a></span></li><li><span><a href="#Get-properties" data-toc-modified-id="Get-properties-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get properties</a></span></li><li><span><a href="#Filter-properties" data-toc-modified-id="Filter-properties-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Filter properties</a></span></li><li><span><a href="#How-do-they-connect-from-each-other?" data-toc-modified-id="How-do-they-connect-from-each-other?-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>How do they connect from each other?</a></span></li></ul></div>

# All wikidata properties

In [1]:
## wikidata property lookup table 

# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#All properties with descriptions and aliases and types
SELECT ?property ?propertyTypeLabel ?propertyLabel ?propertyDescription ?propertyAltLabel WHERE {
  ?property wikibase:propertyType ?propertyType.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  
}
ORDER BY (xsd:integer(STRAFTER(STR(?property), "P")))"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [2]:
from pandas.io.json import json_normalize

results = get_results(endpoint_url, query)
all_wikidata_properties = json_normalize(results['results']['bindings'])

# get results 
all_wikidata_properties = all_wikidata_properties[['property.value',
                                                   'propertyTypeLabel.value',
                                                   'propertyLabel.value',
                                                   'propertyDescription.value',
                                                   'propertyAltLabel.value']]

all_wikidata_properties.columns = ['property_ID', 'DataType', 'property_name','description','examples']

# change formats
all_wikidata_properties['property_ID'] = all_wikidata_properties['property_ID'].str[31:]
all_wikidata_properties['DataType'] = all_wikidata_properties['DataType'][0][26:]

all_wikidata_properties.head()

,property_ID,DataType,property_name,description,examples
0,P6,WikibaseItem,head of government,"head of the executive power of this town, city...","president, chancellor, mayor, prime minister, ..."
1,P10,WikibaseItem,video,"relevant video. For images, use the property P...","animation, media, gif, trailer (Commons)"
2,P14,WikibaseItem,traffic sign,"graphic symbol describing the item, used at th...","highway shield, shield, highway marker, motorw..."
3,P15,WikibaseItem,route map,image of route map at Wikimedia Commons,"schema, highway map, map of route, metro map, ..."
4,P16,WikibaseItem,highway system,system (or specific country specific road type...,"transport network, network of routes, part of ..."


In [3]:
len(all_wikidata_properties)

8823

So as we can see, wikidata has totally 8798 types of properties/relations.

# Get wikidata ID

In [4]:
TERM_LIST = '../01_make_matching_list/matching_list.csv'
WIKIDATA_DICT = './wikidata_id.csv'
WIKIDATA_PROPERTIES = './wikidata_properties.txt'

In [32]:
import pandas as pd 

matching_list = pd.read_csv(TERM_LIST,dtype={'wiki_title': str})

pagetitle_qid_dict = pd.read_csv(WIKIDATA_DICT, delimiter='\t', header=None, index_col=0).to_dict()[1]
matching_list['QID'] = matching_list['wiki_title'].map(pagetitle_qid_dict)

In [6]:
matching_list

,term,annotation,df,wiki_title,QID
0,aperture z-scan experiments,Process,scienceie,NaN,NaN
1,1560nm femtosecond laser pulses,Material,scienceie,NaN,NaN
2,optical-chopper,Material,scienceie,NaN,NaN
3,vibrational combination states,Process,scienceie,NaN,NaN
4,non-radiative processes,Process,scienceie,NaN,NaN
...,...,...,...,...,...
756416,mmds hyper cable,TECH,wikidump,NaN,NaN
756417,parley p pratt,TECH,wikidump,NaN,NaN
756418,de havilland pt-24,TECH,wikidump,NaN,NaN
756419,victorin-hippolyte jasset,TECH,wikidump,NaN,NaN


In [7]:
# target titles that we are going to find their QID
titles = matching_list[matching_list.wiki_title.notna() & matching_list.QID.isna()].wiki_title.values
titles 

array(['Ω−ωf', 'Monoamine neurotransmitter', 'Monosodium glutamate', ...,
       'Torsus Praetorian', 'Dictaphone', 'Subcontrabass flute'],
      dtype=object)

In [40]:
len(titles)

51350

In [8]:
import requests 
import urllib
import re 
from bs4 import BeautifulSoup

def get_wikidata_id(term):
    
    encoded_term = urllib.parse.quote(term)

    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=pageprops&ppprop=wikibase_item&redirects=1&titles={encoded_term}"
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser').get_text()
    
    wikidata_id = re.findall('wikibase_item\"\:\"(.*)?\".*', soup)
    
    if wikidata_id != []:
        return wikidata_id[0]

In [9]:
# find wikidata ID for all terms
from tqdm import tqdm


not_found = []
for title in tqdm(titles):
    wikidata_id = get_wikidata_id(title)
    if wikidata_id:
        pagetitle_qid_dict.update({title: wikidata_id})
    else:
        not_found.append(title)

100%|██████████| 51353/51353 [4:04:47<00:00,  3.50it/s]  


In [10]:
not_found

['Ω−ωf', 'Category:Streaming music services', 'Muscle fibres']

In [11]:
# remove the entities without wikidata ID
titles = [title for title in titles if title not in not_found]

In [12]:
# check disambiguation page
def is_disambiguation_page(wikidata_id):
    
    url = "https://www.wikidata.org/wiki/" + wikidata_id
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    
    div = soup.find("div", {"class": "wikibase-entitytermsview-heading-description"}).text
    
    return div == 'Wikimedia disambiguation page'

In [13]:
QIDs = [pagetitle_qid_dict[title] for title in titles]

In [14]:
# check disambiguation page for list terms
to_remove = []

for title, wikidata_id in tqdm(zip(titles, QIDs)):
    if is_disambiguation_page(wikidata_id):
        to_remove.append(title)

51350it [6:56:42,  2.05it/s]


In [15]:
to_remove

['APAP',
 'PAP',
 'IDM',
 'P11',
 'Progestogen (medication)',
 'Aza',
 'Doca',
 'SNP',
 'TMP',
 'Ciba',
 'AVP',
 'PHE',
 'Pyrrolidone',
 'OCS',
 'NE',
 'DEP',
 'SCL',
 'SIL',
 'TR',
 'LTR',
 'DRA',
 'U937',
 'U1',
 'GR',
 'A10',
 'Dorsal',
 'Enhancer',
 'Receptor',
 'AP1',
 'L1',
 'L2',
 'Pag',
 'Promoter',
 'Promoter',
 'Oct',
 'SP1',
 'Tata',
 'CSF',
 'Jy',
 'CRE',
 'Transcript',
 'Or',
 'PR',
 'Fos',
 'Jun',
 'TRE',
 'Rex',
 'Rex',
 'Mnl',
 'Lmp1',
 'Daudi',
 'Induction',
 'C1',
 '9S',
 'ORF',
 'RAC',
 'P50',
 'P105',
 'Construct',
 'Mab',
 'P95',
 'Seb',
 'Jak',
 'SP1',
 'DCS',
 'P65',
 'Peer',
 'IBR',
 'EWG',
 'Dimer',
 'Factor',
 'IL-9',
 'D10',
 'Bik',
 'Bax',
 'Bak',
 'IL-3',
 'Launch Complex 2',
 'OCT1',
 'Cofactor',
 'Oct',
 'Gata',
 'M44',
 'M67',
 'Receptor',
 'STAT',
 'SFS',
 'IL-10',
 'E7',
 'Silencer',
 'ID',
 'AGS',
 'LCR',
 'Zip',
 'H5',
 'Raz',
 'YT',
 'ILS',
 'RAF (disambiguation)',
 'Alpha subunit',
 'EPO',
 'IL-13',
 'Gyre (disambiguation)',
 'IL-5',
 'Mek',
 'Lyn'

In [16]:
len(to_remove)/len(titles)

0.0197078870496592

In [17]:
# # remove disambiguation page from list entities
# for key in to_remove:
#     dict_pagetitle_qid.pop(key)

In [31]:
pd.DataFrame([(k,v) for k,v in pagetitle_qid_dict.items()]).to_csv(WIKIDATA_DICT, header=None, index=False, sep='\t')

# Get properties 

In [19]:
import numpy as np

def retrieve_value_P(P):
    for i in range(len(all_wikidata_properties)):
        if P == all_wikidata_properties['property_ID'][i]:
            return all_wikidata_properties['property_name'][i]


def retrieve_value_Q(Q):    
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=labels&languages=en&ids={Q}"
    json_response = requests.get(url).json()
    entities = json_response.get('entities')
    
    entity = entities.get(Q)
    if entity:
        labels = entity.get('labels')
        if labels:
            en = labels.get('en')
            if en:
                value = en.get('value')
                return value 


def get_all_ItemProperties(wikidata_item, wikidata_id):
    url = "https://www.wikidata.org/w/api.php?action=wbgetclaims&format=json&entity="+wikidata_id
    json_response = requests.get(url).json()

    properties = [*json_response.get('claims').values()]
    
    res = np.empty(shape=[0, 3])
    
    for p in properties:   
        for d in p: 
            dict_ = d['mainsnak']
        
            # ignore if not a wikibase item
            if dict_['datatype'] != 'wikibase-item' or dict_['snaktype'] != 'value':
                continue 
                
            # replace all the wikidataItem ID by wikidataItem name          
            property_value = retrieve_value_Q(dict_['datavalue']['value']['id'])
            
            if property_value is None:
                continue
            
            # find property value in the previous lookup table
            property_ = retrieve_value_P(dict_['property'])
            
            row_to_append = [wikidata_item, property_, property_value]
  
            res = np.append(res, [row_to_append], 0)  
  
    return res 

In [20]:
## example
# get_all_ItemProperties(wikidata_item='algorithm', wikidata_id='Q8366') 

In [57]:
# find related pairs for all items in our dictionary
if WIKIDATA_PROPERTIES:
    related_pairs = np.loadtxt('./wikidata_properties.txt', delimiter='\t', dtype='str')
else:
    related_pairs = np.empty(shape=[0, 3])

In [33]:
for item, Qid in tqdm(pagetitle_qid_dict.items()):
    if item not in related_pairs[:,0]:
        
        list_to_append = get_all_ItemProperties(item, Qid)
        related_pairs = np.vstack((related_pairs,list_to_append))

100%|██████████| 43943/43943 [11:48:53<00:00,  1.03it/s]   


In [34]:
# check how many of them do have properties

len(np.unique(related_pairs[:,0]))/len(pagetitle_qid_dict.keys())

0.8052477072571286

In [56]:
# save all of them
np.savetxt(WIKIDATA_PROPERTIES, related_pairs, delimiter='\t', fmt='%s')

# Filter properties

In [58]:
# the most frequent wikidata properties for our terms
unique_properties, counts_properties = np.unique(related_pairs[:,1], return_counts=True)

In [59]:
pd.set_option('max_row',1000)

property_df = pd.DataFrame(list(zip(unique_properties, counts_properties)),
                           columns =['property', 'count'])

property_df = property_df.sort_values(by=['count'], ascending=False)
property_df.head(100)

,property,count
449,instance of,28572
799,subclass of,28122
353,found in taxon,12257
397,has part,9012
834,topic's main category,8902
251,described by source,8763
619,part of,7462
116,biological process,7120
262,different from,5450
287,drug used for treatment,3400


In [60]:
# filter only the properties that are useful to us 
property_toremove = ['described by source',
                     'country',
                     'winner',
                     'owner of',
                     'contains administrative territorial entity',
                     'discoverer or inventor',
                     'language used',
                     'language of work or name',
                     'participant',
                     'headquarters location',
                     'indexed in bibliographic review',
                     'cast member',
                     'award received',
                     'founded by',
                     'chairperson',
                     'diplomatic relation',
                     'country of origin',
                     'located in the administrative territorial entity',
                     'LiverTox likelihood score',
                     'publisher',
                     'safety classification and labelling',
                     'has works in the collection',
                     'practiced by',
                     'medical examinations',
                     'operating system',
                     'affiliation',
                     'owned by',
                     'programming language',
                     'developer',
                     'platform',
                     'shares border with',
                     'has edition or translation',
                     'history of topic',
                     'site of astronomical discovery',
                     'copyright license',
                     'afflicts',
                     'child',
                     'nominated for',
                     'official language',
                     'occupation',
                     'on focus list of Wikimedia project',
                     "topic's main template",
                     'characters',
                     'educated at',
                     'doctoral student',
                     'given name',
                     'country of citizenship',
                     'is a list of',
                     'pregnancy category',
                     'notable work',
                     'place of birth',
                     'languages spoken, written or signed',
                     'influenced by',
                     'sport',
                     'family name',
                     'creator'
                    ]

In [61]:
related_pairs_filtered = np.array([pair for pair in related_pairs 
                                   if pair[1] not in property_toremove 
                                   and pair[1] in property_df.property.values[:100].tolist()])

In [62]:
len(np.unique(related_pairs_filtered[:,0]))/len(pagetitle_qid_dict.keys())

0.7956671142161437

# How do they connect from each other?

In [ ]:
# knowledge graph visualization